In [1]:
# !pip install selenium
# !pip install tika
# !pip install olefile
from selenium import webdriver
import pandas as pd
import numpy as np
from tika import parser
from bs4 import BeautifulSoup
import requests
# import olefile # hwp파일 전체 안읽어지는 문제 발생
driver = webdriver.Chrome('./chromedriver.exe')

In [147]:
# 방법1, 게시글 각각의 url을 읽어온 뒤, pdf 링크를 따라 크롤링한다. - 비효율적인 방법(중단)
last_page = 2 # 46
pages = np.arange(1,last_page +1)

url_list = []

for page in pages:
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
    driver.get(url)
    
    li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet')
    for li_tag in li_tags:
        url_list.append(li_tag.get_attribute('href'))

# driver.close()        
url_list

['http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059831&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059657&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059656&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059210&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059055&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10058795&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10058554&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10058032&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10057918&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10057652&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=100

In [151]:
# xpath로 복사붙여넣기했는데도 왜 오류가나지...(9/1 미해결, 한은 viewer 창에서 텍스트가 html에 나타나지 않아서 크롤링 불가 판단)

driver.get(url_list[0])
driver.find_element_by_css_selector('span.icon-viewer').click()
driver.switch_to.window(driver.window_handles[1])
driver.find_element_by_css_selector('#div_page > p:nth-child(4)')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#div_page > p:nth-child(4)"}
  (Session info: chrome=85.0.4183.83)


In [2]:
# 방법2-1,자료(pdf)를 로컬에 다운받고 pdf를 변환한다.
# 먼저 url을 가져온 뒤, url에서 자료를 받는다.

driver = webdriver.Chrome('./chromedriver.exe')
last_page = 2 # 마지막 페이지: 46
pages = np.arange(1,last_page +1)

url_list = []

for page in pages:
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
    driver.get(url)
    
    li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet')
    for li_tag in li_tags:
        url_list.append(li_tag.get_attribute('href'))

# driver.close()        
url_list

['http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059831&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059657&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059656&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059210&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10059055&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10058795&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10058554&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10058032&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10057918&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=10057652&menuNo=200761&pageIndex=1',
 'http://www.bok.or.kr/portal/bbs/B0000245/view.do?nttId=100

In [3]:
# 방법2-2,자료(pdf)를 로컬에 다운받고 pdf를 변환한다. (보존용)
# url 가져올 필요 없이 바로 다운로드 받는다.
driver = webdriver.Chrome('./chromedriver.exe')
last_page = 2 # 46
pages = np.arange(1,last_page +1)

# options = webdriver.ChromeOptions()
# options.add_argument("C:/Users/user/MBP_minutes")
# driver = webdriver.Chrome(chrome_options=options) 로컬 지정장소에 저장하기... 실패

for page in pages:
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
    driver.get(url)
    
    li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet')
    n_li_tags = len(li_tags)
    for idx in range(n_li_tags):
        li_tag = li_tags[idx]
        li_tag.click()
        temp = driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').click()
        driver.back()
        li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet') # 페이지가 새로 로드되면 li_tags를 잃어버리므로 다시 할당
#         li_tags.clear()
#         driver.implicitly_wait(10)
# driver.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)"}
  (Session info: chrome=85.0.4183.83)


In [105]:
# 방법2-2,자료(pdf)를 로컬에 다운받고 pdf를 변환한다.
# url 가져올 필요 없이 바로 다운로드 받는다.
driver = webdriver.Chrome('./chromedriver.exe')
last_page = 46 # 46
pages = np.arange(1,last_page +1)

# options = webdriver.ChromeOptions()
# options.add_argument("C:/Users/user/MBP_minutes")
# driver = webdriver.Chrome(chrome_options=options) 로컬 지정장소에 저장하기... 실패

for page in pages:
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
    driver.get(url)
    
    li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet')
    n_li_tags = len(li_tags)
    for idx in range(n_li_tags):
        li_tag = li_tags[idx]
        li_tag.click()
        name = driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').text
        name = driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').text
        if name.split('.')[1] == 'hwp':
            driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(2) > a:nth-child(1)').click()
        else:
            driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').click()
        driver.back()
        li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet') # 페이지가 새로 로드되면 li_tags를 잃어버리므로 다시 할당
#         li_tags.clear()
#         driver.implicitly_wait(10)
# driver.close()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#board > div > dl > dd > div > ul > li:nth-child(2) > a:nth-child(1)"}
  (Session info: chrome=85.0.4183.83)


In [4]:
# hwp -> text로 변환시키기

#1)로컬 파일의 파일명 리스트를 가져온다.

import os

path = "C:/Users/user/Downloads/MPB_minutes(1)/pdfs/"
file_list = os.listdir(path)
file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_hwp = [file for file in file_list if file.endswith(".hwp")]

file_list = file_list_pdf + file_list_hwp
print(file_list[:10])

['2005년 제12차 금통위 의사록 - PDF파일.pdf', '2005년 제13차 금통위 의사록 - PDF파일.pdf', '2005년 제14차 금통위 의사록 - PDF파일.pdf', '2005년 제17차 금통위 의사록 - PDF파일.pdf', '2005년 제19차 금통위 의사록 - PDF파일.pdf', '2005년 제20차 금통위 의사록 - PDF파일.pdf', '2005년 제21차 금통위 의사록 - PDF파일.pdf', '2005년 제24차 금통위 의사록 - PDF파일.pdf', '2005년 제26차 금통위 의사록 - PDF파일.pdf', '2005년 제27차 금통위 의사록 - PDF파일.pdf']


In [5]:
from tika import parser

file_dict = {}

i = 1
for file in file_list:
    raw = parser.from_file('C:/Users/user/Downloads/MPB_minutes(1)/pdfs/{}'.format(file))
    file_dict[file] = raw['content']
    
    i += 1
    if i % 5 == 0:
        print(i,'번째/',len(file_list),'파일 변환 완료')
    break

2020-09-02 09:25:34,373 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [7]:
file_dict

{'2005년 제12차 금통위 의사록 - PDF파일.pdf': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n130.1.17.56-minute12.hwp\n\n\n- 1 -\n\n金融通貨委員會 議事錄\n\n2005년도 제12차 회의\n\n1. 일    시     2005년 6월 9일 (목)  \n\n2. 장    소     금융통화위원회 회의실\n\n3. 출석위원     박    승  의 장 (총 재)\n\n김 태 동  위 원 \n\n이 성 태  위 원 (부총재)\n\n김 종 창  위 원\n\n강 문 수  위 원\n\n이 덕 훈  위 원\n\n이 성 남  위 원\n\n4. 결석위원     없     음\n\n5. 참 여 자     이 상 용  감사                 정 규   부총재보\n\n               박 재 환  부총재보             김 수 명  부총재보\n\n               이  균  부총재보             정 해 왕  금융경제연구원장\n\n               김 재 천  조사국장             이 주 열  정책기획국장\n\n               김 수 호  금융시장국장         김 두 경  발권국장\n\n               이 광 주  국제국장             정 이 모  금융통화위원회실장\n\n               이 광 준  공보실장\n\n6. 회의경과\n\n〈의안 제21호 ― 통화정책방향〉\n\n(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제68호 - ｢국내외 경제\n\n동향｣, 국제국장이 보고 제69호 - ｢외환․국제금융 동향｣, 그리고 금융시장국장이 \n\n보고 제70호 -｢금융시장 동향｣에 대하여 보고하 음(보고내용 : 별첨)\n\n\n\n- 2 -\n\n(２) 본회의에서는 의장이 ｢한국은행법｣ 제28조에 의거하여 의안 제21호 - 

In [15]:
import pandas as pd
df = pd.DataFrame(file_dict.values())

In [30]:
df.to_csv('MPB_minutes.csv', encoding='utf-8-sig', header=False, index=False)

# df 만들때 columns 설정하면 해결될 수 있음

In [8]:
import pandas as pd

pd.read_csv('MPB_minutes.csv')

\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n130.1.17.56-minute12.hwp\n\n\n- 1 -\n\n金融通貨委員會 議事錄\n\n2005년도 제12차 회의\n\n1. 일    시     2005년 6월 9일 (목)  \n\n2. 장    소     금융통화위원회 회의실\n\n3. 출석위원     박    승  의 장 (총 재)\n\n김 태 동  위 원 \n\n이 성 태  위 원 (부총재)\n\n김 종 창  위 원\n\n강 문 수  위 원\n\n이 덕 훈  위 원\n\n이 성 남  위 원\n\n4. 결석위원     없     음\n\n5. 참 여 자     이 상 용  감사                 정 규   부총재보\n\n               박 재 환  부총재보             김 수 명  부총재보\n\n               이  균  부총재보             정 해 왕  금융경제연구원장\n\n               김 재 천  조사국장             이 주 열  정책기획국장\n\n               김 수 호  금융시장국장         김 두 경  발권국장\n\n               이 광 주  국제국장             정 이 모  금융통화위원회실장\n\n               이 광 준  공보실장\n\n6. 회의경과\n\n〈의안 제21호 ― 통화정책방향〉\n\n(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제68호 - ｢국내외 경제\n\n동향｣, 국제국장이 보고 제69호 - ｢외환․국제금융 동향｣, 그리고 금융시장국장이 \n\n보고 제70호 -｢금융시장 동향｣에 대하여 보고하 음(보고내용 : 별첨)\n\n\n\n- 2 -\n\n(２) 본회의에서는 의장이 ｢한국은행법｣ 제28조에 의거하여 의안 제21호 - ｢통\n\n화정책방향｣을 상정하 음\n\n(３) 위원 토의내용\n\n   ｢국내외 경제동향｣과 관련하여, 일부 위원은 최근 소비재판매액 증가세가 \n\n다소 둔화되고 있으나 GDP 상의 소비와 소비재판매액 간에는 다소 괴리가 있을 \n\n수 있는 데다 소비와 접한 관련이 있는 서비스업활동지수 증가세가 확대되고 \n\n있는 점을 감안할 때 소비회복이 진행중인 것으로 보이며, 기계류 내수출하의 부\n\n진에도 불구하고 내수용 기계류수입이 호조를 보이고 있어 설비투자도 비관적으\n\n로 볼 상황은 아닌 것으로 생각된다는 견해를 나타내었음\n\n   동 위원은 서울 강남에서 시작된 부동산가격 상승이 다른 지역으로 확산될 \n\n기미가 있는 것으로 보인다며 아파트 거래량이 늘어나는 것은 추가적인 가격상승\n\n의 가능성이 높음을 시사하는 것이 아닌지 물었으며, 이에 대해 관련부서에서는 \n\n서울 강남과 판교 주변지역 등 일부 개발호재가 있는 지역을 중심으로 가격상승\n\n이 나타나고 있으나 서울 강북 등 여타 지역은 현재까지 별다른 향을 받지 않\n\n고 있는 것으로 보이며 거래량도 계절성을 감안할 때 2002년이나 2003년에 비해 \n\n많은 수준은 아니라고 설명하 음\n\n   또한 동 위원은 정부가 여러 차례에 걸쳐 부동산시장 안정대책을 내놓았음\n\n에도 불구하고 오히려 정책의 대상이 되는 지역을 중심으로 가격상승이 지속되고 \n\n있어 규제위주의 부동산정책이 실효성이 없음을 보여주는 것으로 생각된다는 견\n\n해를 밝혔으며, 이에 대해 관련부서에서는 외부기관의 분석결과를 참조하는 한편 \n\n내부적으로 과거사례 등을 통해 분석해 본 결과 시장의 내성이 커지고 있는 상황\n\n에서 규제위주의 부동산대책에는 한계가 있으며 공급을 늘려 수급을 안정시키는 \n\n방향의 대책이 바람직한 것으로 생각된다고 답변하 음\n\n   한편 동 위원은 부동산문제에 대해서는 미시적 정책이 우선이라고 하면서 \n\n즉각적인 향력을 발휘하는 미시적 정책과 달리 금리정책의 경우 어느 정도의 \n\n시차를 두고 부동산시장에 향을 미칠 것으로 보는지 물었으며, 이에 대해 관련\n\n부서에서는 금리를 변경하면 곧바로 주택담보대출 등 비용 측면에 향을 미치므\n\n로 금리정책이 부동산시장에 미치는 향은 투자나 소비에 미치는 향에 비해서\n\n\n\n- 3 -\n\n는 시차가 짧을 가능성이 있으며 일부 개발호재가 있는 지역을 중심으로 부동산\n\n가격 상승이 나타나고 있는 현 단계에서 무차별적인 효과를 가진 금리정책으로 \n\n대응할 경우 상당한 무리가 따를 수 있는 것으로 생각된다고 답변하 음\n\n   다른 일부 위원은 최근의 부동산가격 상승이 주로 국지적인 특수성에 기인\n\n한 측면이 있으나 대전, 대구 등 지방 대도시에서도 가격상승이 나타나고 있다는 \n\n점에서 전국적인 확산 가능성을 배제할 수 없다고 언급하고, 토지공사가 집계한 \n\n아파트 거래량이 3월 이후 크게 늘어난 반면 일부 민간기관이 조사하여 발표한 \n\n거래량지수는 하락하는 모습을 보이고 있는 이유를 물었음\n\n   이에 대해 관련부서에서는 토지공사의 거래량 통계는 거래가 일어난 시점\n\n에 비해 한두 달 늦은 등기시점에 집계되는 실제 거래량 통계인 반면 민간기관의 \n\n거래량지수는 부동산중개업소를 대상으로 3개월 이후 시장상황에 대한 주관적인 \n\n전망을 조사한 결과라는 점에서 차이가 나타나는 것으로 보인다고 설명하 음\n\n   동 위원은 외부기관의 조사결과 최근 고유가로 인해 소비를 줄 거나 줄이\n\n겠다고 응답한 가구수가 상당 비중을 차지했다는 점을 인용하면서 통계 숫자만으\n\n로는 확인이 어려운 민간의 소비행태 등에 대해 좀더 면 한 분석이 필요한 것으\n\n로 생각된다는 의견을 개진하 으며, 이에 대해 관련부서에서는 고유가가 실물경\n\n제에 미치는 향을 소비 등 각각의 부문별로 살펴보는 것은 부문간 상호작용이 \n\n일어나는 측면을 간과할 수 있다는 점에서 정확성 면에서 본다면 연립방정식 체\n\n계인 거시계량경제모형을 통해 파악하는 것이 바람직할 것으로 생각된다고 설명\n\n하 음\n\n   또한 동 위원은 미국경제의 회복세와 중국경제의 고성장 이면에 양국의 부\n\n동산시장 상황이 버블 붕괴의 위험을 보이고 있다는 점을 언급하면서 이것이 현\n\n실화될 경우 우리나라 경제에 미치는 향은 어떠할 것으로 보는지 물었음\n\n   이에 대해 관련부서에서는 미국이나 중국의 부동산 버블 붕괴는 미국이나 \n\n중국 경제에 악 향을 주어 우리 경제에도 간접적인 향을 미친다고 볼 수 있겠\n\n으나, 버블 형성 여부에 대한 의견이 분분한 가운데 일부 지역을 중심으로 급등한 \n\n부동산가격이 하락하더라도 미국이나 중국 경제에 큰 충격을 미칠 정도는 아니라\n\n는 것이 일반적인 관측이라고 설명하 음\n\n\n\n- 4 -\n\n   또 다른 일부 위원은 현행 동향보고회의에서는 실물경제와 금융 부문으로 \n\n나뉘어 보고가 이루어지고 있으나 통화총량과 소비, 투자 등 총수요 사이의 관계\n\n에 대한 분석도 심도 있게 다루어질 필요가 있다는 의견을 개진하 으며, 이에 대\n\n해 관련부서에서는 거시계량경제모형 내에 통화․금융 부문이 포함되어 있어 소\n\n비, 투자 등 실물변수에 미치는 향이 적절히 감안되고 있다며 다만, 단기적으로\n\n는 뚜렷한 관계가 나타나지 않는 경우가 많아 월중 보고시에는 생략되는 경우가 \n\n많다고 설명하 음 \n\n   일부 위원은 미국의 경우 경상수지적자와 재정적자가 동시에 나타나는 쌍\n\n둥이적자 상황을 계속 이어가기는 어려울 것이라는 전망이 지배적이며 중국의 경\n\n우 금융기관 부실채권, 고정환율제 등 시스템 측면의 불안정성으로 인해 고도성장\n\n의 지속 가능성 여부에 대한 의문이 제기되고 있다면서, 이처럼 미국과 중국 경제\n\n에 내재되어 있는 취약성이 우리 경제에 미칠 향에 대해 철저히 검토해야 할 \n\n단계에 와 있는 것으로 생각된다는 견해를 피력하 음\n\n   동 위원은 경제의 자유도와 개방도가 높아질수록 안정성(stability)과 안전성\n\n(safety)이 위협받는 상황에서 보다 적극적인 대응이 필요한 것으로 생각

In [ ]:
contents -> raw['content']

행별 자료 하나씩